In [ ]:
#NOTES FOR PLOTTING
#sources for plotting
#https://seaborn.pydata.org/tutorial/categorical.html
#https://flowingdata.com/2019/03/06/women-men-timeuse/

#add expected delay based on length and CAT per bridge --> 
#delaytime per bridge is delaytime(length) * prob_breakingdown(Cat)
#chose different scenario's --> Chose bridges breaking down? 

#traffic data is in ADTT (the total volume of vehicle traffic in both directions on a section of road for a year divided by 365 days)
# Still, the AADT gives a very good indication about the importance of a road, and about the number of truck passages per day or per year, which can be translated to the number of tonnes transported via that road per year.
#dividing the AADT by the number of lanes provides insight into how busy each lane is, and where we can expect most congestion

#To DO FROM NOW
#1. resolve error in summing
#2. combine delay time with traffic data --> laura script
#3. find a way to visualise the characteristics of the 10 most vulnerable/critical bridges with plotly or see seaborn thing

In [11]:
#STEP 1: import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import plotly as plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

%matplotlib inline
import matplotlib.pyplot as plt
from scipy import ndimage
import seaborn as sns

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.tools.set_credentials_file(username='itjallingii', api_key='hiFqLEhfwSwdDlsj9lT8')



In [44]:
#STEP 2 - RUN THIS LINE OF CODE FOR LOADING IN THE DATA
# Create a list of all the LRP's of the bridges
df_all = pd.read_csv('data_clean_all.csv')
df_n1 = pd.read_csv('Data_clean_N1_Dhaka_Chittagong.csv')
df_n1_traffic = pd.read_csv('N1_data_traffic.csv')
road_roadnames = pd.read_csv('_roadnames_list.csv')
road_roadnames_list = list(road_roadnames.columns.values)

df_all_n1 = df_all[df_all["Road"]=="N1"]

#create empty float colums for summing average delay time
df_all["average_delay"] = np.nan
df_n1["average_delay"] = np.nan
#rename some columns in the traffic data for panda operations
df_n1_traffic = df_n1_traffic.rename(columns={'End Chainage': 'End_Chainage', 'No Lanes': 'No_Lanes','Start Chainage': 'Start_Chainage','Link no': 'Link_No','Link Name':'Link_Name',"Link Length": "Link_Length","Road name":"Road_Name"})
df_n1_traffic

,Unnamed: 0,Road_Name,Link_No,Link_Name,Start LRP,Start Offset,Start_Chainage,End LRP,End Offset,End_Chainage,...,Utility,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized Total,Non Motorized Total,Total Traffic
0,0,N1,N1-1,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0.000,LRPS,822,0.822,...,359.0,2229.5,2744.0,417.0,222.5,988.5,0,19236.0,1211.0,20447.0
1,2,N1,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,...,238.0,1851.0,2980.0,398.0,232.0,889.0,0,18236.0,1121.0,19357.0
2,3,N1,N1-3,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,...,306.0,1649.5,2211.5,1061.0,73.5,1169.0,0,15866.5,1242.5,17109.0
3,5,N1,N1-4,Shimrail - Katchpur (Left)N2 (Left),LRPS,7181,7.181,LRP009,260,8.763,...,332.5,1763.5,2923.5,1152.5,364.0,1304.0,0,15998.5,1668.0,17666.5
4,7,N1,N1-5,Katchpur - Madanpur (Left)N105 (Left),LRP009,260,8.763,LRP012,439,11.936,...,297.0,1842.0,3117.0,1148.5,364.0,1304.0,0,22566.0,1668.0,24234.0
5,9,N1,N1-6,Madanpur - Langalband (Left)Z1061 (Left),LRP012,439,11.936,LRP013,3411,15.935,...,297.0,1842.0,3117.0,1148.5,364.0,1304.0,0,22566.0,1668.0,24234.0
6,11,N1,N1-7,Langalband - Mograpara Chowrasta (Left)Z1089 (...,LRP013,3411,15.935,LRP013,7520,20.044,...,297.0,1842.0,3117.0,1148.5,364.0,1304.0,0,22566.0,1668.0,24234.0
7,13,N1,N1-8,Mograpara(Int.with Z1089)-Meghna Bridge West E...,LRP013,7520,20.044,LRP022,1935,23.564,...,399.5,2049.0,1242.0,212.0,34.0,163.5,0,14489.0,197.5,14686.5
8,15,N1,N1-9,Meghna Bridge Satrt-Bhaberchar (Left) z1063 (L...,LRP022,1935,23.564,LRP031,162,30.936,...,399.5,2049.0,1242.0,212.0,34.0,163.5,0,14489.0,197.5,14686.5
9,17,N1,N1-10,Bhaberchar(Int.with Z1063)-Daudkandi Bridge (L...,LRP031,162,30.936,LRP033,3664,36.271,...,399.5,2049.0,1242.0,212.0,34.0,163.5,0,14489.0,197.5,14686.5


In [65]:
#STEP 3 - RUN THIS LINE OF CODE FOR LINKING THE ROAD DATA WITH THE TRAFFIC (PER SEGMENT DATA)

#INCLUDE RIGHT DATA  HERE

df_all_n1["Road_segment_no"] =""
df_all_n1["Road_segment_label"] =""
df_selected = df_all_n1
df_selected["Road_segment_no"] =""
df_selected["Road_segment_label"] =""

df_all_n1 = df_all_n1[:-1] #_---> something with pre rome?

#fasterscript
for row_number in range(len(df_selected)):
    chainage = df_selected.Chainage.iloc[row_number,]
    road_number = df_selected.Road.iloc[row_number]
    
    all_segments_in_road_df = df_n1_traffic[df_n1_traffic["Road_Name"] == road_number ]
    
    for segments in range(len(all_segments_in_road_df)):
        segment_start_chainage = all_segments_in_road_df.Start_Chainage.iloc[segments,]
        segment_end_chainage = all_segments_in_road_df.End_Chainage.iloc[segments,]
        segment_no = all_segments_in_road_df.Link_No.iloc[segments,]
        segment_label = all_segments_in_road_df.Link_Name.iloc[segments,]
        if chainage >= segment_start_chainage:
            if chainage < segment_end_chainage:       
                df_selected.Road_segment_no.loc[row_number,] = segment_no
                df_selected.Road_segment_label.loc[row_number,] = segment_label    

df_all_n1 = df_selected

#iterate through all road segments and get values for that segment
#for i in road_roadnames_list2:
#    single_segment_road_df = df_n1_traffic[df_n1_traffic["Road_Name"] == "N1"]

#    for row_traffic in range(len(single_segment_road_df)):
#        chainage_begin_of_segment = single_segment_road_df.Start_Chainage.iloc[row_traffic,]
#        chainage_end_of_segment = single_segment_road_df.End_Chainage.iloc[row_traffic,]
#        Road_segment_label = single_segment_road_df.Link_Name.iloc[row_traffic,]
#        Road_segment_no = single_segment_road_df.Link_No.iloc[row_traffic,]

 #       #iterate through all LRP points (rows) and set values for that segment
 #       single_road_df = df_all[df_all["Road"] == "N1"]

  #      for row_road in range(len(single_road_df)):
  #          if single_road_df.Chainage.iloc[row_road,] >= chainage_begin_of_segment:
  #              if single_road_df.Chainage.iloc[row_road,] < chainage_end_of_segment:        
  #                  single_road_df.Road_segment_label.loc[row_road,] = Road_segment_label
  #                  single_road_df.Road_segment_no.loc[row_road,] = Road_segment_no
  #          else:
  #              pass

In [68]:
#STEP 4 Determine vulnerability of bridges based on their scenario likelihood --> SEE REPORT FOR EXPLANATION
scenario = {'Scenario': [1,2,3,4,5,6,7,8], 'A': [0,0,0,0,0,0,0.05,0.10], 
            'B': [0,0,0,0,0.05,0.05,0.10,0.20],'C':[0,0,0,0.10,0.10,0.20,0.20,0.40],'D':[0,0.05,0.10,0.20,0.20,0.40,0.40,0.80],
            'Scenario_likelihood': [0,0.40,0.30,0.10,0.075,0.050,0.05,0.025]}
scenario_df = pd.DataFrame(data=scenario)
scenario_df = scenario_df.drop(scenario_df.index[0])

scenario_df["A_overall"] = scenario_df.Scenario_likelihood *scenario_df.A
scenario_df["B_overall"] = scenario_df.Scenario_likelihood *scenario_df.B
scenario_df["C_overall"] = scenario_df.Scenario_likelihood *scenario_df.C
scenario_df["D_overall"] = scenario_df.Scenario_likelihood *scenario_df.D

sum_row = {col: scenario_df[col].sum() for col in scenario_df}
scenario_df_sum = pd.DataFrame(sum_row, index=["Total"])
scenario_df = scenario_df.append(scenario_df_sum)
#---------> A_overall total is vulnerability of bridge

In [73]:
##STEP 5 - RUN THIS LINE OF CODE FOR SYNCHRONIZATION OF BRIDGE DATA TO TRAFFIC DATA
#iterate through all road segments and get values for that segment
road_list = ["N1"]
index = 0

#change df_n1_traffic to the whole traffic file for all in the code here
df_n1_traffic["Total_A"] = ""
df_n1_traffic["Total_B"] = ""
df_n1_traffic["Total_C"] = ""
df_n1_traffic["Total_D"] = ""
df_n1_traffic["segment_vulnerability"] = ""
df_n1_traffic["segment_vulnerability_length"] = ""

for road in road_list: #road_roadnames_list:
    #load in all the roads
    #change df_road to df_all ---> needs to be processed
    df_road = df_all_n1
    segment_list = df_road["Road_segment_no"].unique()
    segment_list = segment_list.tolist()
    
    for segments in segment_list:
        #make small dataframe to count bridges
        df_segment = df_road[df_road["Road_segment_no"] == segments]
        Bridge_count = df_segment.groupby(["Cat"]).count()
        #number_types = Bridge_count.Road.count()
        Bridge_count = Bridge_count["Road"]
        categories = Bridge_count.index.values.tolist()
        #number_types = Bridge_count.Road.count()
        
        #count number of bridges per category and check if they are they are there
        if 'A' in categories:
            total_A_Bridges = Bridge_count['A']
        else:
            total_A_Bridges = 0
        if 'B' in categories:
            total_B_Bridges = Bridge_count['B']
        else:
            total_B_Bridges = 0
        if 'C' in categories:
            total_C_Bridges = Bridge_count['C']
        else:
            total_C_Bridges = 0
        if 'D' in categories:
            total_D_Bridges = Bridge_count['D']
        else:
            total_D_Bridges = 0 
            
        segment_vulnerability = total_A_Bridges * scenario_df.iloc[7]['A_overall'] + total_B_Bridges * scenario_df.iloc[7]['B_overall'] + total_C_Bridges * scenario_df.iloc[7]['C_overall'] + total_D_Bridges * scenario_df.iloc[7]['D_overall'] 
        
        #store total bridges in segment traffic file       
        df_n1_traffic.Total_A.loc[index,] =  total_A_Bridges
        df_n1_traffic.Total_B.loc[index,] =  total_B_Bridges
        df_n1_traffic.Total_C.loc[index,] =  total_C_Bridges
        df_n1_traffic.Total_D.loc[index,] =  total_D_Bridges
        df_n1_traffic.segment_vulnerability.loc[index,] = segment_vulnerability
        #df_n1_traffic.segment_vulnerability_length = df_n1_traffic["segment_vulnerability"] / df_n1_traffic["Link_Length"]
        
        index = index + 1
df_n1_traffic

,Unnamed: 0,Road_Name,Link_No,Link_Name,Start LRP,Start Offset,Start_Chainage,End LRP,End Offset,End_Chainage,...,Cart,Motorized Total,Non Motorized Total,Total Traffic,Total_A,Total_B,Total_C,Total_D,segment_vulnerability,segment_vulnerability_length
0,0,N1,N1-1,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0.000,LRPS,822,0.822,...,0,19236.0,1211.0,20447.0,0,0,0,0,0,
1,2,N1,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,...,0,18236.0,1121.0,19357.0,1,0,0,0,0.005,
2,3,N1,N1-3,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,...,0,15866.5,1242.5,17109.0,1,0,0,0,0.005,
3,5,N1,N1-4,Shimrail - Katchpur (Left)N2 (Left),LRPS,7181,7.181,LRP009,260,8.763,...,0,15998.5,1668.0,17666.5,0,0,0,0,0,
4,7,N1,N1-5,Katchpur - Madanpur (Left)N105 (Left),LRP009,260,8.763,LRP012,439,11.936,...,0,22566.0,1668.0,24234.0,2,1,0,0,0.02625,
5,9,N1,N1-6,Madanpur - Langalband (Left)Z1061 (Left),LRP012,439,11.936,LRP013,3411,15.935,...,0,22566.0,1668.0,24234.0,2,1,0,0,0.02625,
6,11,N1,N1-7,Langalband - Mograpara Chowrasta (Left)Z1089 (...,LRP013,3411,15.935,LRP013,7520,20.044,...,0,22566.0,1668.0,24234.0,1,3,0,0,0.05375,
7,13,N1,N1-8,Mograpara(Int.with Z1089)-Meghna Bridge West E...,LRP013,7520,20.044,LRP022,1935,23.564,...,0,14489.0,197.5,14686.5,3,0,0,0,0.015,
8,15,N1,N1-9,Meghna Bridge Satrt-Bhaberchar (Left) z1063 (L...,LRP022,1935,23.564,LRP031,162,30.936,...,0,14489.0,197.5,14686.5,4,0,0,0,0.02,
9,17,N1,N1-10,Bhaberchar(Int.with Z1063)-Daudkandi Bridge (L...,LRP031,162,30.936,LRP033,3664,36.271,...,0,14489.0,197.5,14686.5,5,1,1,0,0.08875,


In [10]:
#setup dataframe to store aggregate data
statistics = {'road_name':[""],'road_vulnerability':[""],'segment_name':[""],'number_A_bridges':[""], 'number_B_bridges':[""],'number_C_bridges':[""],'number_D_bridges':[""],
              'average_bridge_length':[""],'road_length':[""],'average_delay_bridges': [""]}
descriptive_df = pd.DataFrame(data=statistics,index = range(0,885))
#-----> check dimensions of segment data first before starting handling the data

In [ ]:
#goal of this script: calculate delay time caused by each road and store vulnerability in aggregated file
index = 0
road_list = ["N1","N202"]
for i in road_list:
    #load in all the roads
    df_selected = df_all
    print(descriptive_df)
    
    #determine delaytime per bridge
    for row in range(len(df_selected)):
        if df_selected.Cat.iloc[row,] != 0:        
            if df_selected.Length.iloc[row,] < 10:
                if df_selected.Cat.iloc[row,] == "A": 
                    df_selected.average_delay.iloc[row,] = 15 / 60 * scenario_df.iloc[7]['A_overall']                        
                if df_selected.Cat.iloc[row,] == "B": 
                    df_selected.average_delay.iloc[row,] = 15 / 60 * scenario_df.iloc[7]['B_overall']
                if df_selected.Cat.iloc[row,] == "C":
                    df_selected.average_delay.iloc[row,] = 15 / 60 * scenario_df.iloc[7]['C_overall']
                if df_selected.Cat.iloc[row,] == "D":
                    df_selected.average_delay.iloc[row,] = 15 / 60 * scenario_df.iloc[7]['D_overall']
            if df_selected.Length.iloc[row,] < 50:
                if df_selected.Cat.iloc[row,] == "A": 
                    df_selected.average_delay.iloc[row,] = 37.5 / 60 * scenario_df.iloc[7]['A_overall']
                if df_selected.Cat.iloc[row,] == "B": 
                    df_selected.average_delay.iloc[row,] = 37.5 / 60 * scenario_df.iloc[7]['B_overall']
                if df_selected.Cat.iloc[row,] == "C":
                    df_selected.average_delay.iloc[row,] = 37.5 / 60 * scenario_df.iloc[7]['C_overall']
                if df_selected.Cat.iloc[row,] == "D":
                    df_selected.average_delay.iloc[row,] = 37.5 / 60 * scenario_df.iloc[7]['D_overall']   
            if df_selected.Length.iloc[row,] < 200:
                if df_selected.Cat.iloc[row,] == "A": 
                    df_selected.average_delay.iloc[row,] = 67.5 / 60  * scenario_df.iloc[7]['A_overall']
                if df_selected.Cat.iloc[row,] == "B": 
                    df_selected.average_delay.iloc[row,] = 67.5 / 60 * scenario_df.iloc[7]['B_overall']
                if df_selected.Cat.iloc[row,] == "C":
                    df_selected.average_delay.iloc[row,] = 67.5 / 60 * scenario_df.iloc[7]['C_overall']
                if df_selected.Cat.iloc[row,] == "D":
                    df_selected.average_delay.iloc[row,] = 67.5 / 60 * scenario_df.iloc[7]['D_overall'] 
            if df_selected.Length.iloc[row,] > 200:
                if df_selected.Cat.iloc[row,] == "A": 
                    df_selected.average_delay.iloc[row,] = 2.333 * scenario_df.iloc[7]['A_overall']
                if df_selected.Cat.iloc[row,] == "B": 
                    df_selected.average_delay.iloc[row,] = 2.333 * scenario_df.iloc[7]['B_overall']
                if df_selected.Cat.iloc[row,] == "C":
                    df_selected.average_delay.iloc[row,] = 2.333 * scenario_df.iloc[7]['C_overall']
                if df_selected.Cat.iloc[row,] == "D":
                    df_selected.average_delay.iloc[row,] = 2.333 * scenario_df.iloc[7]['D_overall'] 
            #we need to append here with multiple ROADS !!!!
            df_all_n1 = df_selected
            
            #goal of this script --> save data in aggregated file
    #save descriptive data
    descriptive_df.road_name.iloc[index,] = i
        
    #make small dataframe to count bridges
    bridge_count = df_segment.groupby(["Cat"]).count()
        
    #number_types = Bridge_count.Road.count()
    bridge_count = bridge_count["Road"]
    categories = Bridge_count.index.values.tolist()
        
    #count number of bridges per category and check if they are they are there
    if 'A' in categories:     
        descriptive_df.number_A_bridges.loc[index,] = Bridge_count['A']
    else:
        descriptive_df.number_A_bridges.loc[index,] = 0
    if 'B' in categories:
        descriptive_df.number_B_bridges.loc[index,] = Bridge_count['B']
    else:
        descriptive_df.number_B_bridges.loc[index,] = 0
    if 'C' in categories:
        descriptive_df.number_C_bridges.loc[index,] = Bridge_count['C']
    else:
        descriptive_df.number_C_bridges.loc[index,] = 0
    if 'D' in categories:
        descriptive_df.number_D_bridges.loc[index,] = Bridge_count['D']
    else:
        descriptive_df.number_D_bridges.loc[index,] = 0 
        
    #store vulnerability in overall aggrageted dataframe
    descriptive_df.road_vulnerability.iloc[index,] = descriptive_df.number_A_bridges.loc[index,] * scenario_df.iloc[7]['A_overall'] + descriptive_df.number_B_bridges.loc[index,] * scenario_df.iloc[7]['B_overall'] + descriptive_df.number_C_bridges.loc[index,] * scenario_df.iloc[7]['C_overall'] + descriptive_df.number_D_bridges.loc[index,] * scenario_df.iloc[7]['D_overall']      
        
    #determine average bridge lenght and road length
    length_df = df_selected[df_selected["Cat"] != "0"]
    length = length_df["Cat"].count()
    length_1 = df_selected["Cat"].count() - 1    
    
    #get delaytime from all the bridges
    descriptive_df.average_delay_bridges.iloc[index,] = df_selected.average_delay.sum()
    descriptive_df.average_bridge_length.iloc[index,] = length_df.Length.sum() / length
 
    #get roadlength based on chainage from all the bridges
    descriptive_df.road_length.iloc[index,] = df_selected.Chainage.iloc[length_1,]
    #update index to iterate over alll the roads
    index = index + 1 

    print(descriptive_df)

In [71]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[20, 14, 23],
    name='SF Zoo'
)
trace2 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[12, 18, 29],
    name='LA Zoo'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig) 

In [ ]:
#DENSITY PLOTS
#density here sorted
grades=sorted(df.grade.unique())
data=[]
for g in grades:
    #criticality here
    df_grade=df[df.grade==g]
    data.append(
        go.Scattergl(
            x=df_grade.sqft_living15,
            y=np.log(df_grade.price),
            mode='markers',
            text=[f'Living Room Area:{df_grade.at[i, "sqft_living15"]} sq.ft.<br>Grade:{df_grade.at[i, "grade"]}<br>Price:${df_grade.at[i, "price"]}' for i in df_grade.index],
            marker=dict(
                opacity=0.75,
            ),
            name='Grade:'+str(g)
        )
    )

figure = go.Figure(data=data, layout=layout)

ply.iplot(figure)  

In [ ]:
#CHARACTERISTIC CHARTS about top 10 segments and roads
stunting = "SH.STA.STNT.ZS"
wasting = "SH.STA.WAST.ZS"
overweight = "SH.STA.OWGH.ZS"
anemia = "SH.ANM.ALLW.ZS"
exclusive_breastfeeding = "SH.STA.BFED.ZS"
#Add all the indicators in a list
li = [["SH.STA.STNT.ZS"],["SH.STA.WAST.ZS"],["SH.STA.OWGH.ZS"],["SH.ANM.ALLW.ZS"],["SH.STA.BFED.ZS"]]
indi = []
#Get the data in the right format for all the indicators via a for-loop
for i in range(0,5):
    WDI_SSA_I = WDI_SSA[WDI_SSA["Indicator Code"].isin(li[i])]
    #get all the necessary columns
    SSA_10_17 = pd.concat([WDI_SSA_I.loc[:,'Country Name':'Indicator Code'],WDI_SSA_I.loc[:,'2010':'2017']], axis=1)
    #melt the data into a long format
    melted = pd.melt(SSA_10_17, id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"], var_name="Year", value_name="Value")
    #plot indicator over time for one country
    ll = [ 'Sub-Saharan Africa','World']
    #get only the two regions above and sort them
    mc = melted.loc[melted['Country Name'].isin(ll)] 
    mc = mc.sort_values(["Country Name", "Year"])
    #create two traces, for each region 1
    trace1 = go.Bar(
        x = mc[mc["Country Name"] == 'World']["Year"],
        y = mc[mc["Country Name"] == 'World']["Value"],
        name = 'World',
        legendgroup = 'a',
        showlegend = False,
        marker=dict(color=pblue), 
    )
    trace2 = go.Bar(
        x = mc[mc["Country Name"] == 'Sub-Saharan Africa']["Year"],
        y = mc[mc["Country Name"] == 'Sub-Saharan Africa']["Value"],
        name = 'Sub-Saharan Africa',
        legendgroup = 'b',
        showlegend = False,
        marker=dict(color=porange), 
    )
#Add the two traces together and add them to indi
    data = [trace1, trace2]   
    indi.append(data)
#Plot all the bar charts 
fig = tools.make_subplots(rows=1, cols=5, subplot_titles=('Stunting', 'Wasting', 'Overweight', 'Anemia', 'Exclusive Breastfeeding'), print_grid = False)
for i in range(0,5):
    fig.append_trace(indi[i][0], 1, i+1)
    fig.append_trace(indi[i][1], 1, i+1)
fig['layout'].update(height=300, width=900, title='Figure 4 <br>Target 2.2: Indicators for malnutrition from 2010 till 2017 <br> ')
fig['layout']['yaxis1'].update(title='percentage (%)')
py.iplot(fig, filename='malnutrition-indicators')